In [1]:
print('hello world')

hello world


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .appName("spark-etl_nybike_bronze") \
            .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/05/18 23:56:37 WARN Utils: Your hostname, DESKTOP-2ELJ47S resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/18 23:56:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/18 23:56:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df= spark.read.csv('volumes_and_data_services/data_nybike/2013-citibike-tripdata/6_June/201306-citibike-tripdata_1.csv',header=True)

In [ ]:
df.show(10)

+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|genre_enr|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------+
|         695|2013-06-01 00:00:01|2013-06-01 00:11:36|             444|  Broadway & W 24 St|            40.7423543|           -73.98915076|         434.0|     9 Ave & W 18 St|         40.74317449|         

25/05/18 23:56:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
from pyspark.sql.functions import to_timestamp,col
df=df.withColumn('starttime',to_timestamp(col('starttime')))

In [4]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath("src/jobs/pyspark"))

In [5]:
from src.jobs.pyspark.interfaces import *
from src.jobs.pyspark.transformers import *
from pyspark.sql.functions import lit
df = AddColumnWithLiteralValue().run(df,{'column_to_add':{'column_name':'genre_enr','column_value':'FEMMEUILE'}})

Column With value literal initiated


In [ ]:
import uuid
from src.jobs.pyspark.transformers import AddDimensionsForTimes

In [9]:
import uuid
from pyspark.sql.functions import udf,coalesce,when,col,lit,hash,datediff,year,quarter,dayofmonth,month,dayofweek,date_format,concat,to_timestamp,udf,isnull ,when

@udf
def uuid_gen():
    return str(uuid.uuid4())

# df.select('*',uuid_gen().alias('id_uuid')).show(10)
df.withColumn('id_uuid',uuid_gen()).show(10)

+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+--------------------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|             id_uuid|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+--------------------+
|         695|2013-06-01 00:00:01|2013-06-01 00:11:36|             444|  Broadway & W 24 St|            40.7423543|           -73.98915076|         434.0|     9 Ave & W 18 

In [7]:
from src.jobs.pyspark.interfaces import *
from src.jobs.pyspark.transformers import AddDimensionsForTimesv2
from pyspark.sql.functions import *
config={'datetime_column':'starttime'}
# df.show(1)
AddDimensionsForTimesv2().run(df,config).show(5)


Dimensions time column add initiated
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------+----+-----+-------+---+-------+----------+------------+------------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|genre_enr|year|month|quarter|day|weekday|month_name|weekday_name|quarter_name|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------+----+-----+-------+---+-------+--

In [15]:
from src.jobs.pyspark.interfaces import *
from src.jobs.pyspark.transformers import AddUuidToColumnID
from pyspark.sql.functions import *
config={'column_id':'uuid_id'}
# df.show(1)
AddUuidToColumnID().run(df,config).show(5)

Add uuid in to th column: uuid_id
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------+--------------------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|genre_enr|             uuid_id|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+---------+--------------------+
|         695|2013-06-01 00:00:01|2013-06-01 00:11:36|             444|  Broadway & W 24 St|            40.7

In [12]:
transform = AddRideType()

In [18]:
transform.run(df,config={}).show(5)

Ride type add column initiated
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+-------------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|rideable_type|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+-------------+
|         695|2013-06-01 00:00:01|2013-06-01 00:11:36|             444|  Broadway & W 24 St|            40.7423543|           -73.98915076|         434.0|     9 A